# Transformación de datos

In [ ]:
import pse as PseAAC
from pse import AAIndex
import acc as ACC
import AAComposition as AAC

import sys
import os
import pickle
from math import pow
import time
import errno

import const
from util import frequency
from util import get_data
from util import check_args, read_k, write_to_file
from nac import make_kmer_list
from data import index_list
from pse import get_aaindex
import pandas as pd
import arff

# Declaración de variables

In [ ]:
#organismo ="archaea"
organismo ="E_coli"
#organismo="Heterodera"
#organismo="Meloidogyne"
#organismo="Globodera"
#organismo="fusarium_oxysporum"
#organismo="nematoda"
dataset = 1
nombre = ("ds" + str(dataset) + "_" + str(organismo))

r1 = ("Datos/listas/"+ str(organismo) + "/" + str(nombre))
r2 = ("Datos/resultados/"+ str(organismo) + "/" + str(nombre))
r3 = (str(r2) + "/temp")

#Ruta a los archivos 
efectores_csv = pd.read_csv(str (r1) + '/' + str(nombre) + '_efectores.csv', sep = ',')
efectores_fasta = (str (r1) + '/' + str(nombre) + '_efectores.fasta')
no_efectores_csv = pd.read_csv(str (r1) + '/' + str(nombre) + '_no_efectores.csv', sep = ',')
no_efectores_fasta = (str (r1) + '/' + str(nombre) + '_no_efectores.fasta')

#índices fisicoquímicos 
ind_1 =('Hydrophobicity', 'Hydrophilicity', 'Mass')
ind_2 =('Hydrophobicity', 'Hydrophilicity')
ind_3 =('Mass')

tipo = [str(efectores_fasta), str(no_efectores_fasta)]


In [ ]:
efectores_csv

In [ ]:
no_efectores_csv

# Composición de aminoácidos (AAC)

In [ ]:
#AAC
comp = "AAC"

matrix_efec = []
for idx,row in efectores_csv.iterrows(): #iterar por renglones
    sec = row['secuencia']
    seq_efectores = AAC.CalculateAAComposition(sec)
    
    list_1 = []
    for k in seq_efectores.keys():
        list_1.append(seq_efectores[k])    
    
    list_1.append(row['etiqueta'])
    matrix_efec.append(list_1)
    
#nombre del txt
nom = ('ds' + str(dataset) + '_' + str(comp) + '_efectores_' + str(organismo) + '.temp')

#ruta al directorio donde se almacenara            
os.makedirs(str(r3), exist_ok=True)
resultado = ((r3) + '/' + str(nom))


with open(resultado, 'w') as f:    
    print(matrix_efec, file=f) 


In [ ]:
#AAC
comp = "AAC"

matrix_no_efec = []
for idx,row in no_efectores_csv.iterrows(): #iterar por renglones
    sec2 = row['secuencia']
    seq_no_efectores = AAC.CalculateAAComposition(sec2)
    
    list_2 = []
    for k in seq_no_efectores.keys():
        list_2.append(seq_no_efectores[k])    
    
    list_2.append(row['etiqueta'])
    matrix_no_efec.append(list_2) 
    

#nombre del txt
nom = ('ds' + str(dataset) + '_' + str(comp) + '_no_efectores_' + str(organismo) + '.temp')

#ruta al directorio donde se almacenara            
os.makedirs(str(r3), exist_ok=True)
resultado = ((r3) + '/' + str(nom))

with open(resultado, 'w') as f:    
    print(matrix_no_efec, file=f) 

In [ ]:
df_dataset=pd.concat ([pd.DataFrame(matrix_efec),pd.DataFrame(matrix_no_efec)])

In [ ]:
df_dataset

In [ ]:
print (len(matrix_efec[0]))


# Covarianza de auto cruzamiento (ACC)

In [ ]:
comp = "ACC"

for z in 1, 2, 3:    
    if z == 1:
        indice = ind_1
        res_ind = "hidro_mass"

    if z == 2:
        indice = ind_2
        res_ind = "hidro"

    if z == 3:
        indice = [ind_3]
        res_ind = "mass"        

    for x in tipo:
        for ind in indice:
            out = ACC.acc(open(x), k=1, lag=13, theta_type=3,phyche_list=[ind], alphabet=index_list.PROTEIN)


            #Establece la etiqueta
            if x == str(efectores_fasta):
                etiq = "efectores"

            if x == str(no_efectores_fasta):
                etiq = "no_efectores"

            #nombre del txt
            nom = ('ds' + str(dataset) + '_' + str(comp) + '_' + str(res_ind) + '_' + str(etiq) + '_' + str(organismo) + '.temp')

            #ruta al directorio donde se almacenara            
            os.makedirs(str(r3), exist_ok=True)
            resultado = ((r3) + '/' + str(nom))

            with open(resultado, 'w') as f:    
                print(out, file=f) 
                
            print (indice) 
            print (len(out[0]))

# Composición de pseudo aminoácidos (PseAAC)

In [ ]:
#PseAAC
comp = "PseAAC"
k = 1 # no varia k para este algoritmo
w = 0.5 # peso de los índices 
lamada = 21 # número de niveles para el factores de correlación 
alphabet_list = index_list.PROTEIN
e = None
aa = False
theta_type = 2 #parallel

for z in 1, 2, 3:    
    if z == 1:
        indice = ind_1
        res_ind = "hidro_mass"

    if z == 2:
        indice = ind_2
        res_ind = "hidro"

    if z == 3:
        indice = [ind_3]
        res_ind = "mass" 

    for x in tipo:
        for ind in indice:
            out = PseAAC.pseknc(open(x), k, w, lamada, indice, alphabet_list,extra_index_file=e, all_prop=aa, theta_type=theta_type)           


            #Establece la etiqueta
            if x == str(efectores_fasta):
                etiq = "efectores"

            if x == str(no_efectores_fasta):
                etiq = "no_efectores"


            #nombre del txt
            nom = ('ds' + str(dataset) + '_' + str(comp) + '_' + str(res_ind) + '_' + str(etiq) + '_' + str(organismo) + '.temp')

            #ruta al directorio donde se almacenara            
            os.makedirs(str(r3), exist_ok=True)
            resultado = ((r3) + '/' + str(nom))


            with open(resultado, 'w') as f:    
                print(out, file=f) 
                
            print (indice) 
            print (len(out[0]))